# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

## Лабораторная работа 6

In [2]:
import pandas as pd
import numpy as np

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [3]:
recipes = pd.read_csv('recipes_sample.csv', sep = ',')

In [96]:
id_random = np.random.choice(recipes.id, size = 5, replace = False)
minutes_random = np.array(recipes[np.isin(recipes.id, id_random)].minutes)

In [97]:
max_id_random = len(str(id_random.max()))
max_minutes_random = len(str(minutes_random.max()))

In [116]:
string = "{:{fill}{align}{width}}"

print('|', end = '')
print(string.format('id', fill = ' ', align = '^', width = max_id_random + 6), end = '|')
print(string.format('minutes', fill = ' ', align = '^', width = max_minutes_random + 8), end = '|')
print()
print('|' + '-' * (6 + 8 + max_id_random + max_minutes_random + 1) + '|')

for i in range(len(id_random)):
    print('|', end = '')
    print(string.format(id_random[i], fill = ' ', align = '^', width = max_id_random + 6), end = '|')
    print(string.format(minutes_random[i], fill = ' ', align = '^', width = max_minutes_random + 8), end = '|')
    print()

|     id     |  minutes  |
|------------------------|
|   37833    |    20     |
|   327447   |    65     |
|   351440   |    75     |
|   341857   |    120    |
|   212037   |    35     |


2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [2]:
from bs4 import BeautifulSoup

In [3]:
with open(
    r'/Users/nastya/Desktop/Университет/Tobd/03_data_files/steps_sample.xml',
    'r',
    encoding = 'utf-8'    
) as fp:
    steps = BeautifulSoup(fp, 'xml')

In [163]:
def show_info(name, steps, minutes, author_id):
    result = f'"{name.title()}"\n'
    result += '\n'
    for i in range(len(steps)):
        result += f'{i + 1}. {steps[i].capitalize()}\n'
    result += '-' * 10
    result += f'\nАвтор: {author_id}\n'
    result += f'Среднее время приготовления: {minutes} минут\n'
    
    return result

In [166]:
assert (
    show_info(
        name="george s at the cove black bean soup",
        steps=[
            "clean the leeks and discard the dark green portions",
            "cut the leeks lengthwise then into one-inch pieces",
            "melt the butter in a medium skillet , med",
        ],
        minutes=90,
        author_id=35193,
    )
    == '"George S At The Cove Black Bean Soup"\n\n1. Clean the leeks and discard the dark green portions\n2. Cut the leeks lengthwise then into one-inch pieces\n3. Melt the butter in a medium skillet , med\n----------\nАвтор: 35193\nСреднее время приготовления: 90 минут\n'
)

In [4]:
steps_xml = steps.find_all('steps')
id_recipes_xml = steps.find_all('id')

In [5]:
rr = {}
for rec in range(len(id_recipes_xml)):
    rr[id_recipes_xml[rec].text] = (steps_xml[rec].text).split("\n")[1:-1]

In [174]:
recipes_170895 = recipes[recipes.id == 170895]

In [212]:
print(show_info(name = recipes_170895.name.to_string(index = False),
          steps = rr['170895'],
          minutes = recipes_170895.minutes.to_string(index = False),
          author_id = recipes_170895.contributor_id.to_string(index = False))
     )

"Leeks And Parsnips  Sauteed Or Creamed"

1. Clean the leeks and discard the dark green portions
2. Cut the leeks lengthwise then into one-inch pieces
3. Melt the butter in a medium skillet , med
4. Heat
5. Add the garlic and fry 'til fragrant
6. Add leeks and fry until the leeks are tender , about 6-minutes
7. Meanwhile , peel and chunk the parsnips into one-inch pieces
8. Place in a steaming basket and steam 'til they are as tender as you prefer
9. I like them fork-tender
10. Drain parsnips and add to the skillet with the leeks
11. Add salt and pepper
12. Gently sautee together for 5-minutes
13. At this point you can serve it , or continue on and cream it:
14. In a jar with a screw top , add the half-n-half and arrowroot
15. Shake 'til blended
16. Turn heat to low under the leeks and parsnips
17. Pour in the arrowroot mixture , stirring gently as you pour
18. If too thick , gradually add the water
19. Let simmer for a couple of minutes
20. Taste to adjust seasoning , probably an addi

## Работа с регулярными выражениями

In [1]:
import re

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [12]:
pattern = r'(\d+[ ](?:hours|hour|minutes|minute))\s'

In [13]:
re.findall(pattern, ''.join(rr['25082']))

['20 minutes', '10 minutes', '10 minutes', '20 minutes', '30 minutes']

4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [62]:
pattern = r'\Athis[\w\s]*, ?but.+'

In [92]:
recipes.description[recipes.description.str.match(pattern) == True].count()

134

In [94]:
recipes.description[recipes.description.str.match(pattern) == True][:3]

76     this is a great meal eaten the same day ,but e...
183    this was adapted from a recipe i found on the ...
337    this is kind of similar to some of the other v...
Name: description, dtype: object

In [106]:
recipes.description[76]

"this is a great meal eaten the same day ,but even better the next day , if you can wait! add your favourite spices, but try it first as it is and i think that you will enjoy the 'vegetable' taste. good for freezing."

In [107]:
recipes.description[183]

"this was adapted from a recipe i found on the net, but i added julienne onion to the peppers.  this is a meal in itself, or you could have a small slice with a meat dish.  for those that like to have brunch, it's a bit different to your traditional quiche recipes.  if you love cheese, you could add 1/2 cup of your favorite to the egg mixture, then pour over peppers."

In [108]:
recipes.description[337]

'this is kind of similar to some of the other versions out there, but it is the best and easiest i have found'

5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [56]:
rr['72367']

['mix butter , flour , 1 / 3 c',
 'sugar and 1-1 / 4 t',
 'vanilla',
 'press into greased 9" springform pan',
 'mix cream cheese , 1 / 4 c',
 'sugar , eggs and 1 / 2 t',
 'vanilla beating until fluffy',
 'pour over dough',
 'combine apples , 1 / 3 c',
 'sugar and cinnamon',
 'arrange on top of cream cheese mixture and sprinkle with almonds',
 'bake at 350 for 45-55 minutes , or until tester comes out clean']

In [60]:
print(re.sub(r'(\d) +/ +(\d)', r'\1/\2', '\n'.join(rr['72367'])))

mix butter , flour , 1/3 c
sugar and 1-1/4 t
vanilla
press into greased 9" springform pan
mix cream cheese , 1/4 c
sugar , eggs and 1/2 t
vanilla beating until fluffy
pour over dough
combine apples , 1/3 c
sugar and cinnamon
arrange on top of cream cheese mixture and sprinkle with almonds
bake at 350 for 45-55 minutes , or until tester comes out clean


### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [1]:
from nltk.tokenize import sent_tokenize, word_tokenize

In [39]:
from collections import Counter

In [34]:
text_nltk = ' '.join(' '.join(v) for k, v in rr.items())

In [35]:
word_nltk = word_tokenize(text_nltk)

In [43]:
count_word = Counter([i.lower() for i in word_nltk if i.isalpha()])
count_word

Counter({'in': 77311,
         'cup': 6739,
         'butter': 12701,
         'saute': 3139,
         'carrots': 1440,
         'onion': 7064,
         'celery': 1477,
         'and': 146864,
         'broccoli': 713,
         'stems': 261,
         'for': 36836,
         'minutes': 37888,
         'add': 40838,
         'thyme': 917,
         'oregano': 794,
         'basil': 1271,
         'more': 4027,
         'wine': 1508,
         'deglaze': 109,
         'pan': 16798,
         'hot': 5456,
         'chicken': 10860,
         'stock': 1650,
         'reduce': 3352,
         'by': 2828,
         'worcestershire': 469,
         'sauce': 10850,
         'tabasco': 166,
         'smoked': 100,
         'beans': 2396,
         'florets': 87,
         'simmer': 6279,
         'cream': 8506,
         'season': 3290,
         'to': 74844,
         'taste': 3464,
         'drop': 1038,
         'remaining': 7558,
         'piece': 947,
         'stirring': 6693,
         'until': 44673,


In [47]:
len(count_word)

14926

7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [10]:
def nltk_sent_for_recipes(x):
    return sent_tokenize(x)

In [17]:
recipes['description'].fillna('').map(nltk_sent_for_recipes)[:5]

0    [an original recipe created by chef scott mesk...
1    [my children and their friends ask for my home...
2            [these were so go, it surprised even me.]
3    [my sister-in-law made these for us at a famil...
4    [i think a fondue is a very romantic casual di...
Name: description, dtype: object

In [23]:
def len_nltk_sent_for_recipes(x):
    return len(sent_tokenize(x))

In [27]:
recipes['description'].fillna('').map(len_nltk_sent_for_recipes).sort_values(ascending=False)[:5]

18408    76
481      27
22566    24
16296    23
6779     23
Name: description, dtype: int64

In [36]:
ind = recipes['description'].fillna('').map(
    len_nltk_sent_for_recipes).sort_values(ascending=False).index[:5].tolist()
ind

[18408, 481, 22566, 16296, 6779]

In [38]:
recipes.loc[ind]

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
18408,my favorite buttercream icing for decorating,334113,30,681465,2008-10-30,12.0,this wonderful icing is used for icing cakes a...,NaN
481,alligator claws avocado fritters with chipot...,287008,45,765354,2008-02-19,NaN,a translucent golden-brown crust allows the gr...,9.0
22566,rich barley mushroom soup,328708,60,221776,2008-10-03,NaN,this is one of the best soups i've ever made a...,10.0
16296,little bunny foo foo cake carrot cake with c...,316000,68,689540,2008-07-27,14.0,the first time i made this cake i grated a mil...,NaN
6779,chocolate tea,205348,6,428824,2007-01-14,NaN,i wrote this because there are an astounding l...,NaN


8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [41]:
from nltk import pos_tag

In [128]:
def sentence_in_pos_tag(sentence):
    
    string = "{:{fill}{align}{width}}"
    
    word_nltk = word_tokenize(sentence)
    tagged = pos_tag(word_nltk)
    line1, line0 = zip(*tagged)
    
    line_with_pos_tag = []
    len_word = []
    
    len_word.append([len(i) for i in line1])
    for i in range(len(line0)):
        line_with_pos_tag.append(string.format(line0[i], fill = ' ', 
                                               align = '^', width =  len_word[0][i] + 1, end = ''))
    print(''.join(line_with_pos_tag))
    
    if len_word[0][0] == 1:
        print(' ' + ' '.join(line1))
    else:
        print(' '.join(line1))


In [129]:
sentence_in_pos_tag(sentence)

PRP  VBD    DT     NNS      CC  VBD       NNS        RB   
 I omitted the raspberries and added strawberries instead


In [132]:
sentence_in_pos_tag(recipes[recipes.id == 241106].name.values[0])

   JJ      NNS   IN     NNS     VBP   JJ    CC   JJ    NNS  
eggplant steaks with chickpeas feta cheese and black olives
